In [1]:
import polars as pl

# Why use relative addresses?

In this notebook, we will illustrate

1. That relative addresses for loading data files works, but
2. Using absolute addresses for loading data files *will not*.

## Problem 1 - Load the `lat_long_example.csv` file using a relative address.

**Tasks.**
1. Open a terminal, start `nu`, and navigate to the root menu of your first/primary data repository,
2. Use `ls **/*` to get the relative address of `lat_long_example.csv`, and
3. Use `polars` to load and inspect these data using this relative path.

In [7]:
relative_path = "./data/lat_long_examples.csv"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [13]:
!curl https://raw.githubusercontent.com/nicolebraun310/Lab_6_1_actual/refs/heads/main/data/lat_long_examples.csv -o ./sample_data/lat_long_examples.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   206  100   206    0     0   1966      0 --:--:-- --:--:-- --:--:--  1980


In [14]:
(lat_long_examples :=
 spark.read.csv('./sample_data/lat_long_examples.csv', header=True)
)

DataFrame[City 1: string, Lat 1: string, Long 1: string, City 2: string, Lat 2: string, Long 2: string, Distance from Web (km): string]

## Problem 2 - Load the `lat_long_example.csv` file using a absolute address.

**Tasks.**
1. Open a terminal, start `nu`, and navigate to the root menu of *one of your first/primary data repository,
2. Use `glob **/*` to get the absolute address of `lat_long_example.csv`, and
3. Use `polars` to load and inspect these data.

In [15]:
absolute_path = "C: /Users/fx3734qp/OneDrive - Minnesota State/health_survey/data/lat_long_examples.csv"

## Illustrating the problem with absolute addresses

While the relative address in problem 1 points to the data IN THIS COPY of the repo, the absolute address points to the data in EXACTLY one of the copies of the repository. This becomes a problem if (A) anything changes in that repository, or (B) we are working on a different machine.

**Tasks.** To illustrate why this is a problem, do the following.

1. From your first/primary repository commit and push this notebook to GitHub,
2. Fetch and pull this notebook to another local copy of the repository,
3. In your file explorer (Files or Finder), move your first/primary repository into another folder, e.g., make a new folder and drag-and-drop the repo.
4. Rerun the cells in each local copy of the repository and document your findings in the WORD document.

In [29]:
!curl https://raw.githubusercontent.com/nicolebraun310/Lab_6_1_actual/refs/heads/main/data/health_survey.csv -o ./sample_data/health_survey.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  219k  100  219k    0     0   906k      0 --:--:-- --:--:-- --:--:--  909k


In [30]:
(health_survey :=
 spark.read.csv('./sample_data/health_survey.csv', header=True)
)

DataFrame[_c0: string, F1: string, F5: string, F2: string, F1.1: string, F2.1: string, F6: string, F4: string, F3: string, F5.1: string, F1.2: string, F2.2: string, F6.1: string, F2.3: string, F4.1: string, F2.4: string, F5.2: string, F2.5: string, F6.2: string, F1.3: string, F2.6: string, F5.3: string, F4.2: string, F2.7: string, F3.1: string, F2.8: string, F5.4: string, F3.2: string, F1.4: string, F3.3: string, F1.5: string, F5.5: string, F6.3: string, F1.6: string, F5.6: string, F2.9: string, F3.4: string, F4.3: string, F2.10: string, F1.7: string, F6.4: string, F4.4: string, F5.7: string, F3.5: string, F2.11: string]

In [31]:
!curl https://raw.githubusercontent.com/nicolebraun310/Lab_6_1_actual/refs/heads/main/data/ReverseCodingItems.csv -o ./sample_data/ReverseCodingItems.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3162  100  3162    0     0  16601      0 --:--:-- --:--:-- --:--:-- 16554


In [32]:
(ReverseCodingItems :=
 spark.read.csv('./sample_data/ReverseCodingItems.csv', header=True)
)

DataFrame[Question: string, Construct: string, Question # on Qualtrics Survey: string, Needs Reverse Coding?: string, Column Name: string]

In [33]:
from pyspark.sql.functions import col, explode, array, lit
survey_no_id = health_survey.drop('ID')

melted = survey_no_id.selectExpr("stack({0}, {1}) as (variable, value)".format(
    len(survey_no_id.columns),
    ", ".join([f"'{c}', `{c}`" for c in survey_no_id.columns])
))
unique_values = melted.select("value").distinct()
value_list = [row["value"] for row in unique_values.collect()]


In [34]:
reg = {
    'Strongly Agree': 5,
    'Somewhat Agree': 4,
    'Strongly Disagree': 3,
    'Neither Agree nor Disagree': 2,
    'Somewhat Disagree': 1
}

In [35]:
rev = {'Strongly Agree':1,
         'Somewhat Agree':2,
         'Strongly Disagree':3,
         'Neither Agree nor Disagree':4,
         'Somewhat Disagree':5,
        }

In [37]:
q_col = list(filter(lambda c: c != 'ID', health_survey.columns))
print(q_col)


['_c0', 'F1', 'F5', 'F2', 'F1.1', 'F2.1', 'F6', 'F4', 'F3', 'F5.1', 'F1.2', 'F2.2', 'F6.1', 'F2.3', 'F4.1', 'F2.4', 'F5.2', 'F2.5', 'F6.2', 'F1.3', 'F2.6', 'F5.3', 'F4.2', 'F2.7', 'F3.1', 'F2.8', 'F5.4', 'F3.2', 'F1.4', 'F3.3', 'F1.5', 'F5.5', 'F6.3', 'F1.6', 'F5.6', 'F2.9', 'F3.4', 'F4.3', 'F2.10', 'F1.7', 'F6.4', 'F4.4', 'F5.7', 'F3.5', 'F2.11']


In [39]:
health_survey_summary_df = health_survey.selectExpr(
    "_c0",
    "stack({0}, {1}) as (Question, Response)".format(
        len(q_col),
        ", ".join([f"'{col_name}', `{col_name}`" for col_name in q_col])
    )
)


In [44]:
from pyspark.sql.functions import col, when
melted = melted.withColumn("Response_str", col("Response").cast("string"))
melted = melted.withColumn(
    "reg_coding",
    when(col("Response_str") == "Strongly Agree", 5)
    .when(col("Response_str") == "Somewhat Agree", 4)
    .when(col("Response_str") == "Strongly Disagree", 3)
    .when(col("Response_str") == "Neither Agree nor Disagree", 2)
    .when(col("Response_str") == "Somewhat Disagree", 1)
)
melted = melted.withColumn(
    "rev_coding",
    when(col("Response_str") == "Strongly Agree", 1)
    .when(col("Response_str") == "Somewhat Agree", 2)
    .when(col("Response_str") == "Neither Agree nor Disagree", 2)
    .when(col("Response_str") == "Somewhat Disagree", 4)
    .when(col("Response_str") == "Strongly Disagree", 5)
)


In [45]:
joined = melted.join(
    ReverseCodingItems,
    melted["Question"] == ReverseCodingItems["Column Name"],
    how="left"
)

In [48]:
from pyspark.sql.functions import when, split, col
reverse_clean = ReverseCodingItems.drop("Question") if "Question" in ReverseCodingItems.columns else ReverseCodingItems

joined = melted.join(
    reverse_clean,
    melted["Question"] == ReverseCodingItems
    ["Column Name"],
    how="left"
)


In [50]:
from pyspark.sql.functions import split, col, when

# Add q_type and coding columns
joined = joined.withColumn(
    "q_type",
    split(col("Question"), "\\.").getItem(0)
).withColumn(
    "coding",
    when(col("Needs Reverse Coding?") == "Yes", col("rev_coding")).otherwise(col("reg_coding"))
)


In [56]:
import pandas as pd
health_survey_summary_df = joined.groupBy("_c0").pivot("q_type").sum("coding")
health_survey_summary_df = health_survey_summary_df.withColumnRenamed("_c0", "ID")
health_survey_summary_df.limit(5).toPandas()

,ID,F1,F2,F3,F4,F5,F6,ID
0,125,32,46,22,16,26,18,NaN
1,124,37,55,21,16,36,19,NaN
2,7,35,56,20,18,24,21,NaN
3,51,33,52,24,17,24,23,NaN
4,205,39,43,11,18,30,17,NaN
